In [2]:
cmd_line_run = False
if not cmd_line_run:
    %matplotlib inline
collab_mode = False

if collab_mode and not cmd_line_run:
    # set up tensorflow in collab
    %tensorflow_version 2.x
# imports
import tensorflow as tf
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
import numpy as np

import warnings # This ignore all the warning messages
warnings.filterwarnings('ignore')

from random import randint
from os import path
import os
import time

print("Tensorflow version is", tf.__version__, ", device name", tf.test.gpu_device_name())

batch_size = 1024
image_size = (64, 64)
if not os.path.exists('./embedding'):
    os.makedirs('./embedding')

Tensorflow version is 2.1.0 , device name /device:GPU:0


In [ ]:
def prepare_embeddings_from_batch(batch, output_type=tf.float32):
    imgs, attributes = batch
    embeddings = tf.stack(list(attributes.values()))
    embeddings = tf.transpose(embeddings)
    embeddings = tf.cast(embeddings, dtype=output_type)
    return imgs, embeddings

def process_image(img, image_shape):
    img = tf.cast(img, tf.float32)/127.5-1 # IMPORTANT, image's pixels are in the range <-1, 1>
    img = tf.image.resize(img, image_shape)
    return img

def load_image(filename):
    img = tf.io.read_file(filename)
    img = tf.image.decode_jpeg(img)
    return img

def convert_from_output_to_image(images):
    return tf.clip_by_value((images+1)/2, 0, 1)

def display_image_from_dataset(data):
    for batch in data.take(1):
        image, attributes = batch
        img_ = image
        plt.imshow(img_)
        print(img_.shape, np.min(img_), np.max(img_))

def save_generated_image(settings, epoch):
    save_dir = settings.generated_images_path
    if not path.exists(save_dir):
        os.makedirs(save_dir)
    name = path.join(save_dir,
                     'img_{}_{}.png'.format(epoch, get_time()))
    plt.savefig(name)


def show_images(images, epoch, settings, save_images=False, display_images=False):
    print("image pixels range", np.min(images), np.max(images), "std", np.std(images))
    num_of_images = min(10, images.shape[0])
    # (x, y=1)
    plt.figure(figsize=(num_of_images, 1))
    for i in range(num_of_images):
        plt.subplot(1, num_of_images, i + 1)
        img = images[i, :, :, :].numpy() #
        # img = convert_from_output_to_image(img) # images are already converted in gen_step
        # img = (img * 127.5 + 127.5).astype(np.uint8)
        plt.imshow(img)
        plt.axis('off')
    

    if save_images:
        save_generated_image(settings, epoch)
    if display_images and not cmd_line_run:
        plt.show()

def load_dataset(image_shape, preprocess_images=True, shuffle_size=500, seed=101, split=tfds.Split.TRAIN):
    dataset_name = 'celeb_a'
    data = tfds.load(dataset_name, split=split)\
               .shuffle(shuffle_size)
    # for each image return a tuple (image, attributes), ignore 'landmarks'
    if preprocess_images:
        data = data\
            .map(lambda x: (process_image(x['image'], image_shape), x['attributes']))
    else:
        data = data\
            .map(lambda x: (x['image'], x['attributes']))
    return data.batch(batch_size)
train_data = load_dataset(image_size)
test_data = load_dataset(image_size, split=tfds.Split.TEST)

train_count = 162770
test_count = 19962

In [ ]:
kernel_size = 2
strides = 2
initial_filters = 16
input_shape= (64, 64, 3)
outputs = 40
embedding_size = 128
epochs = 5

model_embedding = tf.keras.Sequential([
    tf.keras.layers.Conv2D(
        filters=initial_filters,
        kernel_size=kernel_size,
        strides=strides,
        input_shape=input_shape),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2D(
        filters=initial_filters*2,
        kernel_size=kernel_size,
        strides=strides),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2D(
        filters=initial_filters*4,
        kernel_size=kernel_size,
        strides=strides),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.LeakyReLU(),
    
    tf.keras.layers.Conv2D(
        filters=initial_filters*8,
        kernel_size=kernel_size,
        strides=strides*2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(embedding_size)
], name='embedding')

model_all = tf.keras.Sequential([
    tf.keras.layers.Dense(outputs, activation='sigmoid', input_shape=(embedding_size, ))
], name='all')

# model_embedding.summary()
# model_all.summary()

In [ ]:
def save_model_and_weights(model, name):
    # save model and its weights
    model_json = model.to_json()
    with open("./embedding/{}_structure.json".format(name), "w") as json_file:
        json_file.write(model_json)
    model.save_weights('./embedding/{}_weights.h5'.format(name))
    
    print('saved model: {}'.format(name))

In [ ]:
def train_step(embedding_model, sigmoids_model):
    loss_f = tf.keras.losses.BinaryCrossentropy()
    optimizer = tf.keras.optimizers.Adam(1e-4)
    
    def test(test_x, test_y):
        test_output = sigmoids_model(embedding_model(test_x))
        return loss_f(test_output, test_y)
    
    def train(data, test_data):
        val = []
        print_every_iter = 5
        time_sum = 0.0
        test_x, test_y = prepare_embeddings_from_batch(next(iter(test_data)))
        for epoch in range(epochs):
            iteration = 0
            train_loss = 0.0
            for batch in data:
                s = time.time()
                x, y = prepare_embeddings_from_batch(batch)
                
                @tf.function
                def _train():
                    with tf.GradientTape() as tape:
                        embedding = embedding_model(x)
                        output = sigmoids_model(embedding)
                        train_loss = loss_f(output, y)

                    grads = tape.gradient(train_loss, 
                                          embedding_model.trainable_variables + sigmoids_model.trainable_variables)
                    optimizer.apply_gradients(zip(grads, 
                                                 embedding_model.trainable_variables + sigmoids_model.trainable_variables))
                _train()
                e = time.time()
                time_sum += e-s
                if (iteration+1)%print_every_iter == 0:
                    progress = (int((iteration+1)*batch_size*100 / train_count))
                    iter_time = (time_sum/print_every_iter)
                    print('epoch ', epoch+1,'progress: ',progress,' %, time ',iter_time)
                    time_sum = 0.0
                iteration+=1
                
            
            test_loss = test(test_x, test_y)
            
            save_model_and_weights(embedding_model, 'embedding_new')
            epoch_res = 'epoch {}/{}, loss {}, test_loss {}'.format(epoch+1, epochs, train_loss, test_loss)
            print(epoch_res)
            val.append(epoch_res)
        return val
            
    return train
        
train_f = train_step(model_embedding, model_all)
res = train_f(train_data, test_data)

with open('representation', 'w') as f:
    f.writelines(res)

In [ ]:
from keras import backend as K

def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
test_iter = iter(test_data)
x, y = prepare_embeddings_from_batch(next(test_iter))
output = model_all(model_embedding(x)).numpy()
o = np.array([1 if x > .5 else 0 for x in output[0]], dtype=np.int8)
print(f1_m(output, y), recall_m(output, y))

In [ ]:
for batch in test_data:
    x, y = prepare_embeddings_from_batch(batch)
    output = model_all(model_embedding(x)).numpy()
    o = np.array([1 if x > .5 else 0 for x in output[0]], dtype=np.int8)
    print(f1_m(output, y), recall_m(output, y))